### And welcome to Week 4, Day 3 - more LangGraph..

In [ ]:
# 匯入型別註釋工具，方便後續型別標註
from typing import Annotated
# 匯入 LangGraph 的狀態圖與起始點常數
from langgraph.graph import StateGraph, START, END
# 匯入訊息處理工具，協助訊息串接
from langgraph.graph.message import add_messages
# 匯入 dotenv，載入環境變數，管理敏感資訊
from dotenv import load_dotenv
# 匯入 IPython 顯示工具，可用於顯示圖片
from IPython.display import Image, display
# 匯入 Gradio，建立互動式網頁介面
import gradio as gr
# 匯入 LangGraph 預設工具節點與條件判斷
from langgraph.prebuilt import ToolNode, tools_condition
# 匯入 requests 套件，進行 HTTP 請求
import requests
# 匯入 os 套件，操作作業系統與環境變數
import os
# 匯入 LangChain OpenAI 聊天模型
from langchain_openai import ChatOpenAI
# 匯入 TypedDict，定義型別字典
from typing import TypedDict


In [ ]:
# Our favorite first step! Crew was doing this for us, by the way.
load_dotenv(override=True)

print(f"OPENAI_API_KEY: {os.getenv('OPENAI_API_KEY')}")
print(f"LANGSMITH_API_KEY: {os.getenv('LANGSMITH_API_KEY')}")
print(f"LANGSMITH_PROJECT: {os.getenv('LANGSMITH_PROJECT')}")
print(f"LANGSMITH_TRACING: {os.getenv('LANGSMITH_TRACING')}")


### First, let's go set up LangSmith!

https://langsmith.com

### Next, here is a useful function in LangChain community:

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

# GoogleSerperAPIWrapper 用於與 Google Serper API 互動，進行網路搜尋
serper = GoogleSerperAPIWrapper()
serper.run("What is the capital of France?")

### Now here is a LangChain wrapper class for converting functions into Tools

In [ ]:
from langchain.agents import Tool

tool_search =Tool(
        name="search",
        func=serper.run,
        description="Useful for when you need more information from an online search"
    )



### Now we can try out the tool the langchain way

In [ ]:
tool_search.invoke("What is the capital of France?")

### And now let's write a tool ourselves

We'll pick a familiar one

In [ ]:
pushover_token = os.getenv("PUSHOVER_TOKEN", "")
pushover_user = os.getenv("PUSHOVER_USER", "")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    # requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

In [ ]:
tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

tool_push.invoke("Hello, me")

### 回到昨天的圖形流程

有一個小改動——將 State 物件由 BaseModel 改為 TypedDict。

當我們實作工具時，通常需要對程式碼做兩個改動：

1. 在呼叫 OpenAI 時，需將工具以 json 格式提供給模型。

2. 回傳結果時，需處理模型回傳 finish_reason=="tool_calls" 的情境，取得呼叫內容、執行對應函式，並回傳結果。

### Bring them together

In [ ]:
tools = [tool_search, tool_push]

In [ ]:
# Step 1: Define the State object
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

In [ ]:
# 建立 OpenAI 聊天模型物件，指定使用 gpt-4o-mini 模型
llm = ChatOpenAI(model="gpt-4o-mini")
# 將工具綁定到聊天模型，讓模型作為指定的工具
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# Step 3: 建立節點（Node）

# 定義 chatbot 節點，負責處理訊息並呼叫 LLM 工具
def chatbot(state: State):
    # 將目前的訊息列表傳給 llm_with_tools，取得模型回應後包裝成新的訊息
    # State接受字典格式，包含 messages 鍵的列表： messages: Annotated[list, add_messages]
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 將 chatbot 節點加入到圖形流程中
graph_builder.add_node("chatbot", chatbot)
# 將工具節點加入到圖形流程中，讓流程能呼叫工具
graph_builder.add_node("tools", ToolNode(tools=tools))

In [ ]:
# Step 4: 建立流程邊（Edge），定義節點間的連接與條件

# 設定 chatbot 節點的條件式邊，當 tools_condition 成立時，流程會轉到 tools 節點
# tools_condition 是一個函數，當 LLM 回覆帶有 "tool_calls" 時會回傳 True，表示需要呼叫工具
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")

# 只要工具被呼叫，流程就會回到 chatbot 節點，讓聊天機器人決定下一步
graph_builder.add_edge("tools", "chatbot")
# 設定流程起始點，從 START 節點進入 chatbot 節點
graph_builder.add_edge(START, "chatbot")

In [ ]:
# Step 5: Compile the Graph
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

### That's it! And, let's do this:

In [ ]:
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]})
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

## 好吧，現在是時候加入記憶功能了！

### 但是等等！

我們有整個圖形在維護狀態並將其附加到狀態中。

為什麼這不能處理記憶？

### 這是理解 LangGraph 的關鍵點

> 超步可以被認為是圖形節點上的一次迭代。並行運行的節點屬於同一超步，而順序運行的節點屬於不同的超步。

一個圖形的 "超步" 代表在代理之間傳遞消息的一次調用。

在 LangGraph 的慣用語中，您調用 invoke 來運行圖形的每個超步；每次交互。

在一個超步內，reducer 自動處理狀態更新，但不會在超步之間處理。

這就是檢查點的作用。

In [ ]:
# 匯入 MemorySaver 類別，用於在記憶體中保存和管理檢查點的狀態
from langgraph.checkpoint.memory import MemorySaver

# 建立 MemorySaver 實例，該實例將用於保存圖形的狀態檢查點
memory = MemorySaver()

In [ ]:
# 步驟 1 和 2：初始化圖形建構器，並指定狀態類別
# StateGraph 是 LangGraph 的核心類別，用於定義狀態圖的結構
# State 是一個型別字典，定義了狀態的結構
graph_builder = StateGraph(State)

# 步驟 3：建立聊天模型並綁定工具
# 初始化 OpenAI 聊天模型，指定使用 gpt-4o-mini 模型
llm = ChatOpenAI(model="gpt-4o-mini")
# 將工具綁定到聊天模型，讓模型能夠使用這些工具進行操作
llm_with_tools = llm.bind_tools(tools)

# 定義 chatbot 節點，負責處理訊息並呼叫 LLM 工具
# state 是一個包含訊息的字典，格式為 {"messages": [...]}
def chatbot(state: State):
    print(state)  # 輸出目前的狀態，方便除錯
    # 呼叫 LLM 工具，並將回應包裝成新的訊息
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 將 chatbot 節點加入到圖形建構器中
# 節點是圖形中的基本單位，代表一個操作或功能
graph_builder.add_node("chatbot", chatbot)
# 將工具節點加入到圖形建構器中，讓流程能夠呼叫工具
graph_builder.add_node("tools", ToolNode(tools=tools))

# 步驟 4：定義節點之間的連接（邊）
# 設定條件式邊，當 tools_condition 成立時，從 chatbot 節點轉到 tools 節點
# tools_condition 是一個函數，當 LLM 回覆帶有 "tool_calls" 時會回傳 True
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")

# 設定無條件邊，從 tools 節點回到 chatbot 節點
# 這樣可以讓聊天機器人決定下一步操作
graph_builder.add_edge("tools", "chatbot")
# 設定起始邊，從 START 節點進入 chatbot 節點
# START 是圖形的預設起始點
graph_builder.add_edge(START, "chatbot")

# 步驟 5：編譯圖形，並指定使用記憶檢查點
# 編譯圖形後，生成一個可執行的圖形物件
# **checkpointer 用於保存和恢復圖形的狀態，這裡使用記憶體檢查點**
graph = graph_builder.compile(checkpointer=memory)
# 使用 Mermaid 圖形語法生成圖形的視覺化表示，並顯示
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

In [ ]:
graph.get_state(config)

In [ ]:
# Most recent first

list(graph.get_state_history(config))

### LangGraph 提供工具將狀態回溯到之前的某個時間點，並從該點分支：

```
config = {"configurable": {"thread_id": "1", "checkpoint_id": ...}}
graph.invoke(None, config=config)
```

這使您能夠構建穩定的系統，並可從任何先前的檢查點恢復和重新運行。

### And now let's store in SQL

### And this is the power of LangGraph.

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

In [ ]:
# 步驟 1 和 2：初始化圖形建構器，並指定狀態類別
# StateGraph 是 LangGraph 的核心類別，用於定義狀態圖的結構
# State 是一個型別字典，定義了狀態的結構
graph_builder = StateGraph(State)

# 步驟 3：建立聊天模型並綁定工具
# 初始化 OpenAI 聊天模型，指定使用 gpt-4o-mini 模型
llm = ChatOpenAI(model="gpt-4o-mini")
# 將工具綁定到聊天模型，讓模型能夠使用這些工具進行操作
llm_with_tools = llm.bind_tools(tools)

# 定義 chatbot 節點，負責處理訊息並呼叫 LLM 工具
# state 是一個包含訊息的字典，格式為 {"messages": [...]}
def chatbot(state: State):
    print(state)  # 輸出目前的狀態，方便除錯
    # 呼叫 LLM 工具，並將回應包裝成新的訊息
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 將 chatbot 節點加入到圖形建構器中
# 節點是圖形中的基本單位，代表一個操作或功能
graph_builder.add_node("chatbot", chatbot)
# 將工具節點加入到圖形建構器中，讓流程能夠呼叫工具
graph_builder.add_node("tools", ToolNode(tools=tools))

# 步驟 4：定義節點之間的連接（邊）
# 設定條件式邊，當 tools_condition 成立時，從 chatbot 節點轉到 tools 節點
# tools_condition 是一個函數，當 LLM 回覆帶有 "tool_calls" 時會回傳 True
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")

# 設定無條件邊，從 tools 節點回到 chatbot 節點
# 這樣可以讓聊天機器人決定下一步操作
graph_builder.add_edge("tools", "chatbot")
# 設定起始邊，從 START 節點進入 chatbot 節點
# START 是圖形的預設起始點
graph_builder.add_edge(START, "chatbot")

# 步驟 5：編譯圖形，並指定使用 SQL 檢查點
# 編譯圖形後，生成一個可執行的圖形物件
# **checkpointer 用於保存和恢復圖形的狀態，這裡使用基於 SQLite 的檢查點**
graph = graph_builder.compile(checkpointer=sql_memory)
# 使用 Mermaid 圖形語法生成圖形的視覺化表示，並顯示
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "3"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()